In [1]:
# Controle de aviso
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew
import os
from utils import get_openai_api_key

In [3]:
openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
print(f'Chave OpenAI {openai_api_key}')

In [4]:
# Definir os agentes existentes
planner = Agent(
    role="Planejador de conteúdo",
    goal="Planeje conteúdo envolvente e factualmente preciso sobre {topic}",
    backstory="Você está planejando um artigo para o blog "
              "sobre o tópico: {topic}. "
              "Você coleta informações que ajudam o "
              "público a aprender algo novo "
              "e tomar decisões informadas. "
              "Seu trabalho é a base para "
              "o redator de conteúdo escrever um artigo sobre este tópico.",
    allow_delegation=False,
    verbose=True
)

writer = Agent(
    role="Escritor de conteúdo",
    goal="Escreva artigos de opinião perspicazes e factualmente precisos "
         "sobre o tema: {topic}",
    backstory="Você está trabalhando em um novo artigo de opinião "
              "sobre o tema: {topic}. "
              "Você baseia sua escrita no trabalho do "
              "planejador de conteúdo, que fornece um esboço "
              "e contexto relevante sobre o tema. "
              "Você segue os objetivos principais e a "
              "direção do esboço "
              "conforme fornecido pelo planejador de conteúdo. "
              "Você também fornece insights objetivos e imparciais "
              "e apoia-os com informações "
              "fornecidas pelo planejador de conteúdo. "
              "Você reconhece em seu artigo de opinião "
              "quando suas declarações são opiniões "
              "em oposição a declarações objetivas.",
    allow_delegation=False,
    verbose=True
)

editor = Agent(
    role="Editor",
    goal="Editar uma determinada postagem do blog para alinhá-la com "
         "o estilo de escrita da organização.",
    backstory="Você é um editor que recebe uma postagem no blog "
              "do redator de conteúdo. "
              "Seu objetivo é revisar a postagem do blog "
              "para garantir que siga as melhores práticas jornalísticas, "
              "forneça pontos de vista equilibrados "
              "ao fornecer opiniões ou afirmações, "
              "e também evite grandes temas polêmicos "
              "ou opiniões quando possível.",
    allow_delegation=False,
    verbose=True
)

tone_analyzer = Agent(
    role='Tone Analyzer',
    goal='Analisar o tom e a linguagem de uma série de postagens',
    verbose=True,
    memory=True,
    backstory=(
        "Você tem um olhar aguçado para entender as nuances da linguagem e do tom. "
        "Você consegue discernir o humor e o estilo do conteúdo com grande precisão."
    ),
    tools=[],  # Não estamos usando ferramentas específicas
    allow_delegation=False
)


In [5]:
# Definir as tarefas existentes
plan = Task(
    description=(
        "1. Priorize as últimas tendências, os principais participantes "
        "e notícias dignas de nota sobre {topic}.\n"
        "2. Identifique o público-alvo, considerando "
        "seus interesses e pontos problemáticos.\n"
        "3. Desenvolva um esboço de conteúdo detalhado, incluindo "
        "uma introdução, pontos-chave e um apelo à ação.\n"
        "4. Inclua palavras-chave de SEO e dados ou fontes relevantes."
    ),
    expected_output="Um documento abrangente de plano de conteúdo "
        "com um esboço, análise de público, "
        "palavras-chave e recursos de SEO.",
    agent=planner,
)

write = Task(
    description=(
        "1. Use o plano de conteúdo e a análise de tom e linguagem para criar uma "
        "postagem do blog sobre {topic}.\n"
        "2. Incorpore palavras-chave SEO naturalmente.\n"
        "3. As seções/legendas estão nomeadas corretamente "
        "de uma maneira envolvente.\n"
        "4. Certifique-se de que a postagem esteja estruturada com uma "
        "introdução envolvente, corpo perspicaz "
        "e uma conclusão resumida.\n"
        "5. Revise erros gramaticais e "
        "alinhamento com a voz da marca.\n"
    ),
    expected_output="Uma postagem de blog bem escrita "
        "em formato markdown, pronta para publicação, "
        "cada seção deve ter 2 ou 3 parágrafos.",
    agent=writer,
)

edit = Task(
    description=("Revise a postagem do blog fornecida para "
                 "erros gramaticais e "
                 "alinhamento com a voz da marca."),
    expected_output="Uma postagem de blog bem escrita em formato markdown, "
                    "pronta para publicação, "
                    "cada seção deve ter 2 ou 3 parágrafos.",
    agent=editor
)

analyze_tone_task = Task(
    description=(
        "Analise o tom e a linguagem das seguintes postagens. "
        "Sua análise deve destacar o humor geral, o estilo e quaisquer padrões notáveis na linguagem. "
        "Esta análise será utilizada pelo escritor de conteúdo para ajustar o tom e a linguagem do novo texto."
    ),
    expected_output='Um relatório detalhado sobre o tom e a linguagem usados nas postagens.',
    tools=[],  # Não estamos usando ferramentas específicas
    agent=tone_analyzer,
    async_execution=False
)


In [6]:
# Atualizar a equipe existente para incluir o novo agente e a nova tarefa
crew = Crew(
    agents=[planner, tone_analyzer, writer, editor],
    tasks=[plan, analyze_tone_task, write, edit],
    verbose=2
)

# Exemplo de postagens para análise
posts = [
    "Provavelmente a coisa mais legal/impressionante em termos de manchetes.\n"
    "É bem similar ao que a OpenAI apresentou ontem, um modelo que você pode ficar "
    "com a câmera ligada mostrando os arredores e conversando com ele, e ele responde em tempo real.\n",
    "Além disso, algo que achei bem bacana foram as melhorias no modo Conversation.\n"
    "A latência diminuiu MUITO, as vozes têm variação de emoção e, com isso, as conversas soam bem mais naturais.\n"
    "Se for possível pedir pra diminuir a velocidade, vai me ajudar MUITO pra praticar idiomas estrangeiros.",
    "Em colaboração nossa com a equipe do Google, que contribuiu com o currículo, ideias, e práticas usando o Google AI Studio.\n"
    "Será um evento gratuito de cinco dias, onde você vai aprender desde técnicas avançadas de prompt até explorar "
    "o ambiente do Google para desenvolver suas próprias ideias e até integrar com a API do Gemini, criando seu próprio chatbot."
]

In [8]:
# Iniciar o processo da equipe
result = crew.kickoff(inputs={"topic": "Automação de redes sociais", "posts": posts})
print(result)

 [DEBUG]: == Working Agent: Planejador de conteúdo
 [INFO]: == Starting Task: 1. Priorize as últimas tendências, os principais participantes e notícias dignas de nota sobre Automação de redes sociais.
2. Identifique o público-alvo, considerando seus interesses e pontos problemáticos.
3. Desenvolva um esboço de conteúdo detalhado, incluindo uma introdução, pontos-chave e um apelo à ação.
4. Inclua palavras-chave de SEO e dados ou fontes relevantes.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Plano de Conteúdo para Automação de Redes Sociais

Introdução:
No mundo digital de hoje, a automação de redes sociais é uma estratégia essencial para empresas e profissionais de marketing alcançarem efetivamente seu público-alvo, economizando tempo e aumentando a eficiência. Neste artigo, vamos explorar as últimas tendências, os principais participantes e notícias dignas de nota sobre automação de redes sociais, identificar o público-alvo e seus interes